In [1]:
import numpy as np
import pandas as pd
import math

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

# Import Data

In [2]:
# Import training and holdout dataframes
train_df = pd.read_csv('kc_house_data_train.csv')
hold_df = pd.read_csv('kc_house_data_test_features.csv')

# Create Features

In [3]:
# Print column names
print(train_df.keys())

Index(['Unnamed: 0', 'id', 'date', 'price', 'bedrooms', 'bathrooms',
       'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
       'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
       'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')


In [4]:
# Function to create house ages
def set_ages(df):
    df['age'] = 2015 - df['yr_built']
    return df

# Create house ages for dataframes
# train_df = set_age(train_df)
# hold_df = set_age(hold_df)

In [5]:
# Function to transform year renovated
def trans_ren(df):
    df['yr_renovated'] = df['yr_renovated'] - 1933
    return df

# Transorm year renovated for dataframes
# train_df = trans_ren(train_df)
# hold_df = trans_ren(hold_df)

In [6]:
# Function for creating year dummy variables
def set_years(df):
    df['year'] = df['date'].apply(lambda x: int(x[0:4]))
    dummies = pd.get_dummies(df['year'], drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    for year in dummies.keys():
        df[year] = df[year] * df['sqft_living']
    df.drop('year', axis=1, inplace=True)
    return df

# Create year dummy variables for dataframes
# train_df = set_years(train_df)
# hold_df = set_years(hold_df)

In [7]:
# Function for creating month dummy variables
def set_months(df):
#     months = {1:'jan', 2:'feb', 3:'mar', 4:'apr', 5:'may', 6:'jun',
#               7:'jul', 8:'aug', 9:'sep', 10:'oct', 11:'nov', 12:'dec'}
    df['month'] = df['date'].apply(lambda x: int(x[4:6]))
    dummies = pd.get_dummies(df['month'], drop_first=True)
#     for key in dummies.keys():
#         if key in months.keys():
#             dummies.rename({key : months[key]}, axis=1, inplace=True)
    df = pd.concat([df, dummies], axis=1)
    for month in dummies.keys():
        df[month] = df[month] * df['sqft_living']
    df.drop('month', axis=1, inplace=True)
    return df

# Create month dummy variables for dataframes
# train_df = set_months(train_df)
# hold_df = set_months(hold_df)

In [8]:
# Function for creating zipcode dummy variables
def set_zipcodes(df):
    dummies = pd.get_dummies(df['zipcode'], drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    for zipcode in dummies.keys():
        df[zipcode] = df[zipcode] * df['sqft_living']
    return df

# Create zipcode dummy variables for dataframes
# train_df = set_zipcodes(train_df)
# hold_df = set_zipcodes(hold_df)

In [9]:
# Function to transform waterfront variable
def trans_waterfront(df):
    df['waterfront'] = df['waterfront'] * df['sqft_living']
    return df

# Transform waterfront variable for dataframes
# train_df = trans_waterfront(train_df)
# hold_df = trans_waterfront(hold_df)

# Apply Features to Dataframes

In [10]:
# Function to create all features for dataframes
def create_featuers(df):
    df = set_ages(df)
    df = trans_ren(df)
    df = set_years(df)
    df = set_months(df)
    df = set_zipcodes(df)
    df = trans_waterfront(df)
    return df

# Create all features for dataframes
train_df = create_featuers(train_df)
hold_df = create_featuers(hold_df)

# Fix Outliers

In [11]:
# Function to transform extreme values
def fix_outliers(df, cols):
    for col in cols:
        if df[col].eq(0).any():
            filt = df[col] == 0
            df[f'{col}_nan'] = np.where(filt, np.nan, df[col])
            std = df[f'{col}_nan'].std()
            mean = df[f'{col}_nan'].mean()
            del df[f'{col}_nan']
        else:
            std = df[col].std()
            mean = df[col].mean()
        value = mean+(5*std)
        df[col] = df[col].apply(lambda x: value if (x>value) else x)
    return df

# Transform extreme values for specified columns dataframes
outlier_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above',
                'sqft_basement', 'sqft_living15', 'sqft_lot15']

train_df = fix_outliers(train_df, outlier_cols)
hold_df = fix_outliers(hold_df, outlier_cols)

# EDA

In [13]:
# # Function to create feature pplots
# def pplot_features(df, features):
#     pp_rows = [features[i:i+4] for i in range(0, len(features), 4)]
#     for row in pp_rows:
#         pp = sns.pairplot(data=df, y_vars=['price'], x_vars=row, kind='reg', height=3)
#     return None

# # Create feature pplots for training df
# pplot_features(train_df, features)

# Split Data Into Train, Test Data

In [14]:
def tt_split(df):
    ignore = ['Unnamed: 0', 'id', 'price', 'date', 'yr_built', 'zipcode',
          'lat', 'long', 'sqft_living', 'sqft_above']
    Y = df['price']
    for col in ignore:
        df = df.drop(columns=col, axis=1)
    x_train, x_test, y_train, y_test = train_test_split(df, Y, random_state=22,test_size=0.2)
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = tt_split(train_df)


In [15]:
hold_df = hold_df.drop(columns=['Unnamed: 0', 'id', 'date', 'yr_built', 'zipcode',
          'lat', 'long', 'sqft_living', 'sqft_above'])

# Run Model

In [16]:
model = sm.OLS(y_train, sm.add_constant(x_train[[key for key in x_train]])).fit()
model.summary()

/Users/eric/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.880
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                     1068.
Date:                Sun, 03 May 2020   Prob (F-statistic):               0.00
Time:                        20:51:40   Log-Likelihood:            -1.8251e+05
No. Observations:               13832   AIC:                         3.652e+05
Df Residuals:                   13737   BIC:                         3.659e+05
Df Model:                          94                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          -2.65e+05   1.58e+04    -16.750      0.000   -2.96e+05   -2.34e+05
bedrooms        -2.1e+04   1587.689    -13.227      0.000   -2.41e+04   -1.79e+04
bathrooms      1.927e+04   2660.248      7.243      0.000    1.41e+04    2.45e+04
sqft_lot          0.7736      0.071     10.898      0.000       0.634       0.913
floors        -4.303e+04   3142.542    -13.692      0.000   -4.92e+04   -3.69e+04
waterfront      234.6488      3.741     62.719      0.000     227.315     241.982
view           4.759e+04   1763.999     26.978      0.000    4.41e+04     5.1e+04
condition      2.316e+04   1929.067     12.004      0.000    1.94e+04    2.69e+04
grade           5.12e+04   1814.920     28.210      0.000    4.76e+04    5.48e+04
sqft_basement   -87.7411      3.676    -23.868      0.000     -94.947     -80.536
yr_renovated     14.0910      3.007      4.686      0.000       8.196      19.986
sqft_living15    42.9967      2.956     14.548      0.000      37.203      48.790
sqft_lot15       -0.3163      0.109     -2.912      0.004      -0.529      -0.103
age             304.5008     64.521      4.719      0.000     178.032     430.970
2015             45.2368      2.899     15.607      0.000      39.555      50.918
2                 4.9794      3.036      1.640      0.101      -0.971      10.930
3                13.6608      2.808      4.865      0.000       8.157      19.165
4                20.3795      2.677      7.611      0.000      15.131      25.628
5                40.7701      3.104     13.135      0.000      34.686      46.854
6                48.9178      3.569     13.706      0.000      41.922      55.914
7                46.4691      3.568     13.025      0.000      39.476      53.462
8                50.6863      3.634     13.950      0.000      43.564      57.809
9                45.2458      3.657     12.372      0.000      38.077      52.414
10               50.3149      3.622     13.893      0.000      43.216      57.414
11               46.4220      3.751     12.377      0.000      39.070      53.774
12               48.2436      3.757     12.839      0.000      40.878      55.609
98002            17.4174      7.393      2.356      0.018       2.926      31.909
98003            15.1344      5.687      2.661      0.008       3.988      26.281
98004           336.6651      4.437     75.877      0.000     327.968     345.362
98005           156.3848      5.586     27.998      0.000     145.436     167.333
98006           154.0375      4.209     36.596      0.000     145.787     162.288
98007           144.4234      7.072     20.422      0.000     130.561     158.285
98008           140.9285      5.431     25.950      0.000     130.283     151.574
98010            53.9115      7.674      7.025      0.000      38.869      68.954
98011            85.6952      6.068     14.123   

# Feature Selection

In [17]:
lasso = Lasso(alpha=0.01, normalize=False)
lasso.fit(x_train, y_train)

y_train_pred = lasso.predict(x_train)
y_pred = lasso.predict(x_test)

In [18]:
train_rmse = metrics.mean_absolute_error(y_train, y_train_pred)
test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print('Training Error: '+ str(train_rmse) )
print('Testing Error: '+ str(test_rmse) )

Training Error: 78772.80508360255
Testing Error: 140701.1835566215


In [20]:
lasso_coef = pd.DataFrame(data=lasso.coef_).T
lasso_coef.columns = x_train.columns
lasso_coef = lasso_coef.T.sort_values(by=0).T

lasso_coef

,floors,bedrooms,sqft_basement,sqft_lot15,sqft_lot,98022,2,98023,98092,3,...,98109,age,98112,98004,98102,98039,bathrooms,condition,view,grade
0,-43027.787932,-21000.592977,-87.741013,-0.316302,0.773613,4.740865,4.97936,6.904438,10.763425,13.6608,...,295.491575,304.500832,312.807885,336.665116,364.097732,467.855588,19266.852258,23155.727036,47588.514666,51199.168558


# Use Model to Forecast

In [23]:
# hold_df = hold_df.drop(columns=['Unnamed: 0', 'id', 'date', 'yr_built', 'zipcode',
#           'lat', 'long', 'sqft_living', 'sqft_above'])

In [24]:
hold_df.keys()

Index([     'bedrooms',     'bathrooms',      'sqft_lot',        'floors',
          'waterfront',          'view',     'condition',         'grade',
       'sqft_basement',  'yr_renovated', 'sqft_living15',    'sqft_lot15',
                 'age',            2015,               2,               3,
                     4,               5,               6,               7,
                     8,               9,              10,              11,
                    12,           98002,           98003,           98004,
                 98005,           98006,           98007,           98008,
                 98010,           98011,           98014,           98019,
                 98022,           98023,           98024,           98027,
                 98028,           98029,           98030,           98031,
                 98032,           98033,           98034,           98038,
                 98039,           98040,           98042,           98045,
                 98052,  

In [35]:
def calc_price(row):
    price = 0
    for index, value in enumerate(row):
        price += value * coef_list
    return price

In [ ]:
def forecast(df, coef):
    df.apply(calc_price)
    return df

# Save Results

# Notes (TESTING)

In [32]:
# # Recreate original dataframe without any added columns
# TESTING = pd.read_csv('kc_house_data_train.csv')

# # Create features list for original dataframe
# TESTING_FEATURES = set_features(TESTING)

# # Run model on original dataframe
# set_model(TESTING, TESTING_FEATURES)

# Unused Functions

In [33]:
# # Function to create target variable
# def set_target(df):
#     target = df['price']
#     return target

# # Create target variable for dataframes
# train_target = set_target(train_df)
# test_target = set_target(test_df)

In [34]:
# # Function for creating grade dummy variables
# def set_grades(df):
#     dummies = pd.get_dummies(df['grade'], prefix='grade', drop_first=True)
#     df = pd.concat([df, dummies], axis=1)
#     return df

# # Create grade dummy variables for dataframes
# train_df = set_grades(train_df)
# holdout_df = set_grades(holdout_df)

In [35]:
# # Function to transform bedrooms outliers
# def bedrooms_trans(df):
#     filt = df['bedrooms'] < 10
#     df['bedrooms'] = np.where(filt, df['bedrooms'], 10)
#     return df

# # Transform bedrooms outliers in dataframes 
# train_df = bedrooms_trans(train_df)
# hold_df = bedrooms_trans(hold_df)

In [36]:
# # Function to transform bathrooms outliers
# def bathrooms_trans(df):
#     low_filt = df['bathrooms'] > .5
#     df['bathrooms'] = np.where(low_filt, df['bathrooms'], .5)
#     high_filt = df['bathrooms'] < 5
#     df['bathrooms'] = np.where(high_filt, df['bathrooms'], 5)
#     return df

# # Transform bedrooms outliers in dataframes 
# train_df = bathrooms_trans(train_df)
# hold_df = bathrooms_trans(hold_df)

In [37]:
# # Function to create regression modeln (smf method)
# def set_model(df, features):
#     formula = 'price~' + '+'.join([f'{ft}' for ft in features])
#     model = smf.ols(formula=formula, data=df).fit()
#     return model.summary()

# # Run model on train dataframe
# set_model(train_df, train_features)

In [26]:
# # Function to create age range dummy variables
# def set_ages(df):
#     year = df['yr_built']
#     # df['age_new'] = np.where(age==2015, 1, 0)
#     df['age_10'] = np.where((year>2004) & (year<2015), 1, 0)
#     df['age_20'] = np.where((year>1994) & (year<2005), 1, 0)
#     df['age_30'] = np.where((year>1984) & (year<1995), 1, 0)
#     df['age_40'] = np.where((year>1974) & (year<1985), 1, 0)
#     df['age_50'] = np.where((year>1964) & (year<1975), 1, 0)
#     df['age_60'] = np.where((year>1954) & (year<1965), 1, 0)
#     df['age_70'] = np.where((year>1944) & (year<1955), 1, 0)
#     df['age_80'] = np.where((year>1934) & (year<1945), 1, 0)
#     df['age_90'] = np.where((year>1924) & (year<1935), 1, 0)
#     df['age_100'] = np.where((year>1914) & (year<1925), 1, 0)
#     df['age_old'] = np.where((year<1915), 1, 0)
#     return df

# # Create age range dummy variables for dataframes
# # train_df = set_ages(train_df)
# # hold_df = set_ages(hold_df)

In [27]:
# # Function to create renovation age range dummy variables
# def set_ren_ages(df):
#     year = df['yr_built']
#     ren_yr = df['yr_renovated']
#     df['not_ren'] = np.where((year!=2015) & (ren_yr==0), 1, 0)
#     df['ren_5'] = np.where((ren_yr>2010), 1, 0)
#     df['ren_10'] = np.where((ren_yr>2005) & (ren_yr<2011), 1, 0)
#     df['ren_15'] = np.where((ren_yr>2000) & (ren_yr<2006), 1, 0)
#     df['ren_20'] = np.where((ren_yr>1995) & (ren_yr<2001), 1, 0)
#     df['ren_25'] = np.where((ren_yr>1990) & (ren_yr<1996), 1, 0)
#     df['ren_30'] = np.where((ren_yr>1985) & (ren_yr<1991), 1, 0)
#     # df['ren_old'] = np.where((ren_yr>0) & (ren_yr<1986), 1, 0)
#     return df

# # Create renovation age range dummy variables for dataframes
# # train_df = set_ren_ages(train_df)
# # hold_df = set_ren_ages(hold_df)

In [23]:
# # Transform grade variable
# df_test = train_df
# df_test['grade_log'] = df_test['grade'].apply(lambda x: math.log(x))

# df_test = train_df
# df_test['grade_exp'] = df_test['grade'].apply(lambda x: math.exp(x))

# df_test = train_df
# df_test['price_log'] = df_test['price'].apply(lambda x: math.log(x))


# test1 = df_test.groupby('grade_exp')['price'].mean()
# fig, ax = plt.subplots()
# ax.scatter(test1.index, test1.values)

In [35]:
# # Function to create features list
# def set_features(df):
#     ignore = ['Unnamed: 0', 'id', 'price', 'date', 'yr_built', 'zipcode',
#               'lat', 'long', 'sqft_living', 'sqft_above']
#     features = list(df.keys())
#     for feature in ignore:
#         if feature in features:
#             features.remove(feature)
#     return features

# # Create features list for dataframes
# features = set_features(train_df)

In [31]:
# # Function to create regression model
# def set_model(df, features):
#     model = sm.OLS(y_train, sm.add_constant(
#         df[[key for key in features]])).fit()
#     return model.summary()
# # Run model on training dataframe
# set_model(train_df, features)

In [ ]:
# # Split into x dataframes and y series for train and test data
# Y = train_df['price']
# X = train_df.drop(['price'], axis=1)
# x_train, x_test, y_train, y_test = train_test_split(
#     X, Y, random_state=22,test_size=0.2)